In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import SGDRegressor
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV 

In [ ]:
import os
import random

folder_path = "C:\\Users\\Sameed\\Desktop\\ML_Overall_Work\\More_Improving_Age_Regression\\Dataset\\UTKFace"
max_images = 2000

# get a list of all files in the folder
files = os.listdir(folder_path)

# determine how many files to delete
num_files_to_delete = len(files) - max_images

# make sure we don't try to delete more files than exist in the folder
num_files_to_delete = min(num_files_to_delete, len(files))

# randomly select the files to delete
files_to_delete = random.sample(files, num_files_to_delete)

# delete the selected files
for file in files_to_delete:
    if(file[0:2] == 22 or file[0:2] == 20 or file[0:2] == 21):
        continue
    os.remove(os.path.join(folder_path, file))


In [ ]:
#Path to the raw images
path_unknown_raw = "C:\\Users\\Sameed\\Desktop\\ML_Overall_Work\\More_Improving_Age_Regression\\Dataset\\UTKFace"

#Get the names of all the images
images_unknown = os.listdir(path_unknown_raw)

#Store the paths of all the cropped images we will save later
paths_unknown_cropped = []

img_dim = 32


In [ ]:
unknown_ages = []
features = []

#Store all the paths for the unknown images
for unknowns in enumerate(images_unknown):
    
    age = ""
    for i in range(0,6):
        if(unknowns[1][i] == "_"):
            break
        else:
            age += unknowns[1][i]

    unknown_ages.append(int(age))

    path = path_unknown_raw + "\\" + unknowns[1]
    image = cv2.imread(path)

    img_resize = cv2.resize(image, (img_dim,img_dim))
    features.append(img_resize.flatten())


In [ ]:
x = np.array(features)
y = np.array(unknown_ages)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:


param_grid = {
    "penalty" : ['l1', 'l2', 'elasticnet', None],
    "eta0" : [0.00001,0.0001,0.001,0.01,0.1]
}

sgd_reg = SGDRegressor()
grid_search = GridSearchCV(sgd_reg, param_grid, cv=None, verbose=2)
grid_search.fit(x,y)




In [ ]:
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

In [ ]:
#Best parameters: {'eta0': 1e-05, 'penalty': 'l1'}

regressor = SGDRegressor(penalty="l1", eta0=0.00001)
regressor.fit(x_train, y_train)


y_pred_sgd = regressor.predict(x_test)
y_pred_sgd2 = regressor.predict(x_train)

mse = mean_squared_error(y_test, y_pred_sgd)
n_r2 = r2_score(y_test, y_pred_sgd)

mse2 = mean_squared_error(y_train, y_pred_sgd2)
n_r22 = r2_score(y_train, y_pred_sgd2)

print("intercept=", regressor.intercept_)
print("coef=", regressor.coef_)

print("Best Parameters: \n")
print(f"Testing: \n R2 = {n_r2} \n MSE = {mse} \n\n Validation: \n R2 = {n_r22} \n Mse = {mse2}" )


regressor = SGDRegressor()
regressor.fit(x_train, y_train)


y_pred_sgd = regressor.predict(x_test)
y_pred_sgd2 = regressor.predict(x_train)

mse = mean_squared_error(y_test, y_pred_sgd)
n_r2 = r2_score(y_test, y_pred_sgd)

mse2 = mean_squared_error(y_train, y_pred_sgd2)
n_r22 = r2_score(y_train, y_pred_sgd2)

print("------------------------------------")
print("\nintercept=", regressor.intercept_)
print("coef=", regressor.coef_)

print("Not Best Parameters: \n")
print(f"Testing: \n R2 = {n_r2} \n MSE = {mse} \n\n Validation: \n R2 = {n_r22} \n Mse = {mse2}" )



In [ ]:

ridge = RidgeCV(alphas=[0.00001,0.0001,0.001,0.01,0.1])

ridge.fit(x, y)



y_pred_sgd = ridge.predict(x_test)
y_pred_sgd2 = ridge.predict(x_train)

mse = mean_squared_error(y_test, y_pred_sgd)
n_r2 = r2_score(y_test, y_pred_sgd)

mse2 = mean_squared_error(y_train, y_pred_sgd2)
n_r22 = r2_score(y_train, y_pred_sgd2)

print("Best alpha:", ridge.alpha_)
print("intercept=", ridge.intercept_)
print("coef=", ridge.coef_)

print(f"Testing: \n R2 = {n_r2} \n MSE = {mse} \n\n Validation: \n R2 = {n_r22} \n Mse = {mse2}" )



In [ ]:

lasso = LassoCV(alphas=[0.00001,0.0001,0.001,0.01,0.1])

lasso.fit(x, y)

y_pred_sgd = lasso.predict(x_test)
y_pred_sgd2 = lasso.predict(x_train)

mse = mean_squared_error(y_test, y_pred_sgd)
n_r2 = r2_score(y_test, y_pred_sgd)

mse2 = mean_squared_error(y_train, y_pred_sgd2)
n_r22 = r2_score(y_train, y_pred_sgd2)

print("Best alpha:", lasso.alpha_)
print("intercept=", lasso.intercept_)
print("coef=", lasso.coef_)

print(f"Testing: \n R2 = {n_r2} \n MSE = {mse} \n\n Validation: \n R2 = {n_r22} \n Mse = {mse2}" )



In [ ]:



elastic = ElasticNetCV(alphas=[0.00001,0.0001,0.001,0.01,0.1])

elastic.fit(x, y)

y_pred_sgd = elastic.predict(x_test)
y_pred_sgd2 = elastic.predict(x_train)

mse = mean_squared_error(y_test, y_pred_sgd)
n_r2 = r2_score(y_test, y_pred_sgd)

mse2 = mean_squared_error(y_train, y_pred_sgd2)
n_r22 = r2_score(y_train, y_pred_sgd2)

print("Best alpha:", elastic.alpha_)
print("intercept=", elastic.intercept_)
print("coef=", elastic.coef_)

print(f"Testing: \n R2 = {n_r2} \n MSE = {mse} \n\n Validation: \n R2 = {n_r22} \n Mse = {mse2}" )



In [37]:
reg = LinearRegression()
reg.fit(x_train, y_train)

y_pred_ls = reg.predict(x_test)
y_pred_ls2 = reg.predict(x_train)

r2_test = r2_score(y_test, y_pred_ls)
mse_test = mean_squared_error(y_test, y_pred_ls)

r2_val = r2_score(y_train, y_pred_ls2)
mse_val = mean_squared_error(y_train, y_pred_ls2)

print("Intercept =", reg.intercept_)
print("Coef =", reg.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )


Intercept = 38.12487146907304
Coef = [-0.04209414  0.07690981 -0.02304046 ... -0.00274011 -0.12711271
  0.0824205 ]
Testing: 
 R2 = -0.9990195731247142 
 MSE = 829.1712797413379 

 Validation: 
 R2 = 1.0 
 Mse = 3.346933820646476e-25


MODEL SCORES 
------------

SGD REGRESSOR:

Vanilla: 

intercept= [-6510559.13356393]

coef= [ 2.85271893e+09  4.04443965e+09  2.10045590e+09 ...  2.14085840e+09
  1.86366217e+09 -4.64352187e+07]
Best Parameters: 

Testing: 

 R2 = -6.136579884729392e+23 

 MSE = 2.545385680392523e+26 

 Validation: 

 R2 = -6.142326687268915e+23 
 
 Mse = 2.465349636023673e+26

 -----------------------------------
 
Best Parameters:  ({'eta0': 1e-05, 'penalty': 'l1'})

intercept= [-1.24776856e+10]

coef= [ 8.92280236e+11  1.69939602e+10 -1.79392353e+11 ... -1.12733220e+12
 -8.94342196e+11 -1.45806076e+12]

Testing: 

 R2 = -1.8805803179858888e+30 

 MSE = 7.800439825025411e+32 

 Validation: 

 R2 = -1.9401894884924392e+30 

 Mse = 7.787351101311386e+32


--------------------------------------------------------------------------
RidgeCV

Best alpha: 0.1

intercept= 32.24971284172287

coef= [-0.03931608  0.05673108  0.03631565 ... -0.01160425 -0.06682718
  0.03073856]

Testing: 

 R2 = 0.9999999997682644 

 MSE = 9.612138510547381e-08 


 Validation: 

 R2 = 0.9999999997500001 

 Mse = 1.003426140097847e-07

----------------------------------
LassoCV

Best alpha: 0.1

intercept= 33.18064129269854

coef= [ 1.03905603e-02  4.30949479e-02  2.17743279e-02 ...  1.24233691e-02
 -4.19654655e-02 -1.54735697e-05]

Testing: 

 R2 = 0.9679533292227248 

 MSE = 13.292605723868451 


 Validation: 

 R2 = 0.9661942649178298 

 Mse = 13.56862975931979

------------------------------
ElasticNetCV

Best alpha: 0.1

intercept= 33.27006779550928

coef= [ 0.00252324  0.043004    0.03099204 ...  0.01641882 -0.05293846
  0.00815768]

Testing: 

 R2 = 0.9743859628176875 

 MSE = 10.624420228463283 

 Validation: 

 R2 = 0.9730442185897158 

 Mse = 10.819259422706844

-----------------------
OLS

Intercept = 38.12487146907304

Coef = [-0.04209414  0.07690981 -0.02304046 ... -0.00274011 -0.12711271
  0.0824205 ]

Testing: 

 R2 = -0.9990195731247142 

 MSE = 829.1712797413379 


 Validation: 

 R2 = 1.0 

 Mse = 3.346933820646476e-25

In [38]:
#Choosing the best model

#SGD
pickle.dump(regressor, open('sgd.pkl', 'wb'))
#RidgeCV
pickle.dump(ridge, open('ridge.pkl', 'wb'))
